<a href="https://colab.research.google.com/github/eiman-ops/AG-News-Topic-Classification-/blob/main/AG_News_Topic_Classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importing the data**

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 9.3MB/s 
     |████████████████████████████████| 194kB 19.6MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 1.2MB 19.6MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
Mounted at /content/gdrive


In [2]:
from fastai.text.all import *
import pandas as pd
from fastbook import *

In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
root_dir = 'gdrive/My Drive/UnpackAi2/AG-News-Topics/'
base_dir = root_dir + 'AG-News-Topics'

Mounted at /content/drive


In [4]:
path = Path('/content/drive/My Drive/UnpackAi2/AG-News-Topics/')
path.ls()

(#3) [Path('/content/drive/My Drive/UnpackAi2/AG-News-Topics/train.csv'),Path('/content/drive/My Drive/UnpackAi2/AG-News-Topics/test.csv'),Path('/content/drive/My Drive/UnpackAi2/AG-News-Topics/Model')]

# **Language model**

### Creating DataLoaders for the Language Model

In [7]:
dl_lm = TextDataLoaders.from_csv(path=path, csv_fname='train.csv' ,
                                 text_col='Title', label_col='Class', is_lm=True, seq_len=72 , bs=64)
dl_lm.show_batch(max_n=3)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,"xxbos xxmaj bangladesh crash to xxmaj mills pace xxbos xxmaj edward xxup c. xxmaj baig : xxmaj personal xxmaj tech - xxmaj express takes digital music a step further , but it 's no ipod ( usatoday.com ) xxbos nbc 's five - ring xxmaj circus xxbos xxmaj jaguar confirms cut in production xxbos xxmaj eisner xxmaj will xxmaj end xxmaj reign at xxmaj disney xxbos xxmaj newest ipod xxmaj from xxmaj","xxmaj bangladesh crash to xxmaj mills pace xxbos xxmaj edward xxup c. xxmaj baig : xxmaj personal xxmaj tech - xxmaj express takes digital music a step further , but it 's no ipod ( usatoday.com ) xxbos nbc 's five - ring xxmaj circus xxbos xxmaj jaguar confirms cut in production xxbos xxmaj eisner xxmaj will xxmaj end xxmaj reign at xxmaj disney xxbos xxmaj newest ipod xxmaj from xxmaj apple"
1,xxmaj first xxmaj round xxbos xxmaj xxunk xxunk raped tens of thousands in xxmaj congo : xxmaj amnesty report ( canadian xxmaj press ) xxbos xxmaj abbott to xxmaj buy xxmaj nutrition xxmaj company xxup eas xxbos xxmaj constellation xxmaj agrees to xxmaj acquire xxmaj robert xxmaj mondavi for \ $ 1 xxmaj billion xxbos xxup hbos cries foul as xxmaj monti ' favours ' bid xxbos xxmaj mutu 's agent hits,first xxmaj round xxbos xxmaj xxunk xxunk raped tens of thousands in xxmaj congo : xxmaj amnesty report ( canadian xxmaj press ) xxbos xxmaj abbott to xxmaj buy xxmaj nutrition xxmaj company xxup eas xxbos xxmaj constellation xxmaj agrees to xxmaj acquire xxmaj robert xxmaj mondavi for \ $ 1 xxmaj billion xxbos xxup hbos cries foul as xxmaj monti ' favours ' bid xxbos xxmaj mutu 's agent hits out
2,payton xxmaj award ( ap ) xxbos xxup ecb sees gradual recovery in eurozone area xxbos xxmaj german swoop on online piracy site nets thousands of user names ( afp ) xxbos xxup abn xxup amro xxmaj leads xxmaj european xxmaj shares xxmaj higher xxbos xxmaj citigroup 's ex - exec xxmaj may xxmaj face xxup sec xxmaj action xxbos xxmaj braves xxmaj edge xxmaj dodgers 6 - 5 xxbos xxmaj bush,xxmaj award ( ap ) xxbos xxup ecb sees gradual recovery in eurozone area xxbos xxmaj german swoop on online piracy site nets thousands of user names ( afp ) xxbos xxup abn xxup amro xxmaj leads xxmaj european xxmaj shares xxmaj higher xxbos xxmaj citigroup 's ex - exec xxmaj may xxmaj face xxup sec xxmaj action xxbos xxmaj braves xxmaj edge xxmaj dodgers 6 - 5 xxbos xxmaj bush preaches


### Creating the Language Model

In [8]:
learn = language_model_learner(
   dl_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [9]:
learn

In [10]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.861618,3.706181,0.423632,40.698093,01:53


In [11]:
learn.unfreeze()
learn.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.357683,3.483979,0.444683,32.589138,01:52
1,3.189121,3.363207,0.457997,28.881655,01:54
2,2.931424,3.331253,0.466356,27.973358,01:53
3,2.579038,3.346513,0.471478,28.403526,01:53
4,2.365739,3.396025,0.470423,29.845221,01:52


Saving the Language Model

In [18]:
learn.save('model',with_opt=True)

Path('/content/drive/My Drive/UnpackAi2/AG-News-Topics/models/model.pth')

In [19]:
learn.save_encoder('model_finetuned')

# **Text Generation**

**Data Classes :  World , Sports , Business , Sci/Tech**


### Generating world-related text

In [24]:
TEXT = "global warming"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [25]:
print("\n".join(preds))

global warming : ' we ' ve learned it Insurance Probe Widens to Six Levels Study : Kids Left in Africa Begged for Change Two Blasts
global warming ' may be warming ' Blogs Fuel Blogs , Blogs , and Blogs Bayern stun Stuttgart , win again British firms see more jobs abroad New


In [ ]:
print("\n".join(preds))

global warming is in new shape Afghan Blasts Kill at Least 15 ; Taliban Claims Responsibility New Windows Planned for 2006 Oil prices jump over China
global warming could deter sale : study Cricket : England beat Sri Lanka in Bangladesh Test ( afp ) Sudan Government Says Rebels Vow to End War


### Generating sports-related text

In [22]:
TEXT = "The game last night"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [23]:
print("\n".join(preds))

The game last night US Airways pilots vote for pay cuts Israeli Forces Kill Palestinian Girl : Killing Girl ( reuters ) U.S . Men Fail to Reach
The game last night Stocks Look to Open Higher ; Oil , CPI Data Weigh Do You Want to Resign ? Scientists Look to Separate Your


In [ ]:
print("\n".join(preds))

The game last night Howard Stern to Join Sirius Satellite Radio UPDATE * As Roma , Italian held in Palermo Yahoo Unveils Traffic Search Tool
The game last night American League Game Summary - Los Angeles at Los Angeles Rice Says He Will Get Disciplined With Storm Is


### Generating business-related text

In [20]:
TEXT = "The stock market"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [21]:
print("\n".join(preds))

The stock market ' on the horizon ' Not That Long Until 2007 China Netcom to Cut Up to 15 , 000 Jobs by 2008 ( reuters )
The stock market returns Stocks Down on Weak Ad Sales , Oil American Greetings Reports 3rd - quarter Profit of \ $ 13.5 Billion Indians Sign


In [ ]:
print("\n".join(preds))

The stock market in a hurry Sharon Seeks to Back Labor Policies , Labor Policies ( reuters ) Three Dutch Police Killed in Blast ( ap )
The stock market comes to a close against Google China 's Yudhoyono Sworn in as President US Airways Seeks Financing Financing Financing New Attitude : Prince


### Generating Sci/Tech-related text

In [26]:
TEXT = "graphics card"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [27]:
print("\n".join(preds))

graphics card industry cools Ballmer : Linux Can Withstand High Oil a Comeback Comeback for Mcgill Park Keeps Winning Run at Oakland Hills
graphics card for free speech software Taylor Charged With Referee Violations Afghan Kidnappers Vow to Keep Taliban In Kabul Rossi clinches podium in Portuguese


In [ ]:
print("\n".join(preds))

graphics card users not guilty Google IPO Close to End Two - way tie ends for Astros Group Urges Local Managers to Take Shelter US
graphics card software for Mac pcs New York Knicks Bush Signs Corporate Tax Cut Bill ( ap ) Windows Media Player 2 Now Available


# **Classifier**

### Creating DataLoaders for the Classifier

In [29]:
path = Path('/content/drive/My Drive/UnpackAi2/AG-News-Topics/')
dl_clas = TextDataLoaders.from_csv(path=path, csv_fname='train.csv' ,
                                 text_col='Title', label_col='Class', seq_len=72 , bs=64)
dl_clas.show_batch(max_n=3)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,category
0,"xxbos xxup at & t 's xxmaj gambit : xxmaj relying xxmaj on xxmaj automation xxmaj to xxmaj make xxmaj up xxmaj for 12 , xxrep 3 0 xxmaj fewer < b > … < / b >",4
1,xxbos xxup football : i xxup will xxup bring xxup the xxup glory xxup glory xxup days xxup back xxup to xxup spurs .. < b > … < / b >,2
2,xxbos xxmaj putin 's xxmaj war xxmaj if xxmaj putin xxmaj refuses to xxmaj change xxmaj course xxmaj russia 's xxmaj future is < b > … < / b >,1


In [31]:
learn = text_classifier_learner(dl_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

### Loading the Language Model 

In [32]:
learn = learn.load_encoder('model_finetuned')

In [33]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.531241,0.442717,0.842500,02:25


In [34]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.442179,0.371486,0.865917,02:24


In [35]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.378211,0.331651,0.881792,02:30


In [36]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.342490,0.326915,0.885083,02:35
1,0.321416,0.320119,0.887375,02:36
2,0.299299,0.315556,0.890958,02:36
3,0.278863,0.316530,0.892708,02:37
4,0.234864,0.322229,0.892292,02:39
5,0.234449,0.324021,0.893250,02:39
6,0.218597,0.337620,0.894208,02:37
7,0.195955,0.345355,0.894000,02:36
8,0.197232,0.348352,0.893750,02:36
9,0.177624,0.349325,0.893208,02:36


### Predictions

World = 1 ,
Sports = 2 , 
Business= 3 ,
Sci/Tech = 4


In [37]:
learn.predict("The basketball game was postponed because all the players got sick")

('2', tensor(1), tensor([8.0088e-06, 9.9940e-01, 4.9919e-04, 9.3463e-05]))

In [47]:
learn.predict("when will world war 3 start")

('1', tensor(0), tensor([0.9086, 0.0080, 0.0181, 0.0653]))

In [48]:
learn.predict("where was the kingdom of kush located")

('4', tensor(3), tensor([0.2728, 0.0184, 0.0570, 0.6518]))

In [39]:
learn.predict("The COVID-19 pandemic is unleashing a new era of change for businesses")

('3', tensor(2), tensor([9.7749e-02, 5.2909e-04, 8.2627e-01, 7.5451e-02]))

In [38]:
learn.predict("fastai is a deep learning library which provides practitioners with high-level components")

('4', tensor(3), tensor([0.0010, 0.0016, 0.2947, 0.7028]))

In [41]:
learn.predict("Coronavirus disease is an infectious disease caused by a newly discovered coronavirus")

('4', tensor(3), tensor([1.3119e-01, 6.0485e-04, 1.8184e-02, 8.5003e-01]))

In [42]:
learn.predict("COVID-19 is Changing the Ecommerce Landscape")

('4', tensor(3), tensor([0.0280, 0.0770, 0.1661, 0.7289]))

In [43]:
learn.predict("what is COVID-19 ")

('2', tensor(1), tensor([0.0105, 0.5508, 0.4166, 0.0221]))

In [44]:
learn.predict("Who is Coronavirus ")

('2', tensor(1), tensor([0.4066, 0.5757, 0.0145, 0.0032]))

In [45]:
learn.predict("What is Coronavirus ")

('2', tensor(1), tensor([0.0364, 0.5309, 0.4216, 0.0110]))

In [46]:
learn.predict("Retail businesses around the world over are being affected by COVID-19")

('3', tensor(2), tensor([6.5184e-04, 1.4512e-06, 9.9438e-01, 4.9621e-03]))

In [49]:
learn.save('classifier',with_opt=True)

Path('/content/drive/My Drive/UnpackAi2/AG-News-Topics/models/classifier.pth')